<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=43b2f0fffd9eba7fcc311e2a469410f1fa63943ba3f359f77ce2e6ffa4a029e1
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2


    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-16 13:05:31
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.41 C
-------------------
Today PnL: -48.86 K (-0.35%)
Current PnL: -26.46 L (-17.09%)
CY Booked + Current PnL: -12.17 L (-7.86%)
-------------------
Total profit:  1.40 L
Total loss:  -27.86 L
-------------------
Total Booked + Current PnL: 14.81 L (11.59%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.16%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 91.10 L (64.79%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.3%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-1.05,-18.16,22.30,0.09,18393.0,-18299.0,82478.0,91.46,35.0,M-SC,2.33,253.0,-0.99,0.59,8.36,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,0.40,9.08,9.72,19.69,20330.0,17418.0,209160.0,-1.49,58.0,X-MC,2.53,78.0,0.86,1.49,16.50,XY25,NTT,AC
39,INDIGOPNTS,1408.00,-1.54,-13.04,15.01,0.02,22785.0,-22758.0,151801.0,131.58,50.0,M-SC,14.80,234.0,-1.00,1.08,32.04,OX40N,NTT,PAINTS
50,MASFIN,398.61,-1.87,-1.90,24.41,22.05,23463.0,-1860.0,96120.0,-15.54,55.0,H-SC,6.90,164.0,-0.08,0.69,39.72,XR,ATH,FINANCE
43,ITC,452.00,0.06,-1.34,12.28,10.78,29116.0,-3216.0,237102.0,-41.15,47.0,X-LC,1.95,5.0,-0.11,1.69,3.83,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,CERA,9475.00,3.22,-30.09,77.22,23.89,94957.0,-52933.0,122970.0,-32.46,40.0,H-SC,10.57,157.0,-0.56,0.88,6.20,OX40N,NTT,CERAMICS
25,FINCABLES,1641.55,2.26,-1.79,111.70,107.91,161965.0,-2644.0,145000.0,-20.15,58.0,M-SC,12.54,216.0,-0.02,1.04,8.11,OX40N,ATH,CABLES
6,ASIANTILES,137.00,2.16,-16.72,118.15,81.67,92302.0,-15687.0,78123.0,7022.22,46.0,L-SC,12.37,271.0,-0.17,0.56,51.29,XR,NTT,CERAMICS
23,DMART,5391.45,1.68,-6.50,38.60,29.60,90088.0,-16212.0,233388.0,-20.29,41.0,X-LC,5.66,19.0,-0.18,1.67,14.27,X40N,ATH,FMCG
61,ROUTE,2227.21,1.52,-47.15,219.89,69.05,154629.0,-62745.0,70321.0,-57.12,52.0,H-SC,22.88,140.0,-0.41,0.50,8.45,SR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RAJOOENG,143.10,-3.11,-35.88,117.74,39.61,77379.0,-36780.0,65720.0,-56.61,34.0,H-SC,28.74,135.0,-0.48,0.47,3.54,AR,ATH,MISC
63,SFL,1287.00,-2.29,-44.28,127.26,26.63,185933.0,-116113.0,146105.0,10.73,25.0,M-SC,11.68,239.0,-0.62,1.04,0.00,XY24,NTT,MISC
32,HCLTECH,1922.01,-2.16,6.93,16.65,24.73,43072.0,16765.0,258689.0,11.49,55.0,X-LC,6.82,7.0,0.39,1.85,23.22,X40,ATH,IT
0,5PAISA,593.00,-2.00,-40.85,90.86,12.90,117437.0,-89245.0,129251.0,89.90,45.0,H-SC,12.02,161.0,-0.76,0.92,7.47,OX40N,NTT,FINANCE
58,RECLTD,446.00,-1.96,-9.10,32.82,20.74,60615.0,-18480.0,184690.0,19.44,28.0,M-MC,7.70,189.0,-0.30,1.32,0.00,XY25,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,2.26,-1.79,111.70,107.91,161965.0,-2644.0,145000.0,-20.15,58.0,M-SC,12.54,216.0,-0.02,1.04,8.11,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.54,-13.04,15.01,0.02,22785.0,-22758.0,151801.0,131.58,50.0,M-SC,14.80,234.0,-1.00,1.08,32.04,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-1.05,-18.16,22.30,0.09,18393.0,-18299.0,82478.0,91.46,35.0,M-SC,2.33,253.0,-0.99,0.59,8.36,OX40N,NTT,DURABLES
66,SIS,528.00,0.14,-24.12,59.81,21.26,50391.0,-26780.0,84252.0,1987.56,50.0,H-SC,5.12,166.0,-0.53,0.60,13.83,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.29,-24.67,50.64,13.47,102870.0,-66528.0,203139.0,-69.09,37.0,H-SC,5.18,158.0,-0.65,1.45,3.29,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.66,6.64,61.53,72.27,111189.0,11259.0,180707.0,-8.35,65.0,M-LC,2.46,99.0,0.1,1.29,13.23,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.66,6.64,61.53,72.27,111189.0,11259.0,180707.0,-8.35,65.0,M-LC,2.46,99.0,0.10,1.29,13.23,XR,NTT,IT
25,FINCABLES,1641.55,2.26,-1.79,111.70,107.91,161965.0,-2644.0,145000.0,-20.15,58.0,M-SC,12.54,216.0,-0.02,1.04,8.11,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-1.63,-4.19,115.76,106.72,136792.0,-5167.0,118169.0,-52.54,34.0,H-SC,3.00,139.0,-0.04,0.84,18.21,AR,ATH,MISC
1,ABB,7934.00,-0.52,-3.54,50.91,45.57,128476.0,-9259.0,252360.0,-37.97,58.0,H-MC,4.55,120.0,-0.07,1.80,7.38,AR,NTT,ELECTRICAL
50,MASFIN,398.61,-1.87,-1.90,24.41,22.05,23463.0,-1860.0,96120.0,-15.54,55.0,H-SC,6.90,164.0,-0.08,0.69,39.72,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.38,-18.25,90.19,55.47,179366.0,-44410.0,198876.0,-27.60,25.0,X-MC,12.93,64.0,-0.25,1.42,1.90,X40N,ATH,FINANCE
53,PAGEIND,51605.08,-0.73,-9.46,41.50,28.11,75675.0,-19060.0,182350.0,-34.38,28.0,X-MC,12.93,55.0,-0.25,1.30,0.00,X40,ATH,APPARELS
75,TMPV,600.00,-0.62,-29.29,73.91,22.98,118060.0,-66163.0,159735.0,-26.74,29.0,X-LC,6.81,3.0,-0.56,1.14,0.47,XY24,NTT,AUTO
11,BATAINDIA,2096.00,-1.19,-38.86,121.13,35.20,95295.0,-49998.0,78672.0,3.10,31.0,X-SC,16.13,92.0,-0.52,0.56,0.00,X40,NTT,FOOTWEAR
33,HINDUNILVR,2922.00,-0.24,-10.99,27.70,13.67,63380.0,-28253.0,228810.0,-17.61,32.0,X-LC,2.14,14.0,-0.45,1.64,8.02,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-0.49,-1.53,26.41,24.48,65495.0,-3861.0,247995.0,-81.32,43.0,X-SC,0.31,86.0,-0.06,1.77,21.25,X40N,NTT,MISC
43,ITC,452.0,0.06,-1.34,12.28,10.78,29116.0,-3216.0,237102.0,-41.15,47.0,X-LC,1.95,5.0,-0.11,1.69,3.83,X40,NTT,FMCG
33,HINDUNILVR,2922.0,-0.24,-10.99,27.70,13.67,63380.0,-28253.0,228810.0,-17.61,32.0,X-LC,2.14,14.0,-0.45,1.64,8.02,XY25,NTT,FMCG
36,ICICIPRULI,790.0,-0.43,6.85,22.35,30.73,47046.0,13493.0,210498.0,-18.17,63.0,X-MC,2.30,75.0,0.29,1.50,20.49,X40,ATH,INSURANCE
78,UNITDSPR,1644.0,0.35,6.33,13.58,20.77,33810.0,14826.0,248970.0,-2.93,57.0,X-MC,2.44,62.0,0.44,1.78,13.69,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.44,-25.63,120.72,64.15,213638.0,-60981.0,176970.0,-56.31,34.0,X-SC,2.60,82.0,-0.29,1.26,0.00,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-1.19,-38.86,121.13,35.20,95295.0,-49998.0,78672.0,3.10,31.0,X-SC,16.13,92.0,-0.52,0.56,0.00,X40,NTT,FOOTWEAR
53,PAGEIND,51605.08,-0.73,-9.46,41.50,28.11,75675.0,-19060.0,182350.0,-34.38,28.0,X-MC,12.93,55.0,-0.25,1.30,0.00,X40,ATH,APPARELS
75,TMPV,600.00,-0.62,-29.29,73.91,22.98,118060.0,-66163.0,159735.0,-26.74,29.0,X-LC,6.81,3.0,-0.56,1.14,0.47,XY24,NTT,AUTO
59,RELAXO,1176.00,-0.33,-47.82,191.88,52.31,145340.0,-69415.0,75745.0,-44.44,41.0,X-SC,6.93,91.0,-0.48,0.54,1.39,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.56,-29.39,102.29,42.84,46953.0,-19104.0,45902.0,-53.72,43.0,X-SC,35.52,83.0,-0.41,0.33,3.85,XY24,NTT,MISC
59,RELAXO,1176.00,-0.33,-47.82,191.88,52.31,145340.0,-69415.0,75745.0,-44.44,41.0,X-SC,6.93,91.0,-0.48,0.54,1.39,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.19,-38.86,121.13,35.20,95295.0,-49998.0,78672.0,3.10,31.0,X-SC,16.13,92.0,-0.52,0.56,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.54,-4.65,30.12,24.08,36461.0,-5898.0,121052.0,-12.39,32.0,X-SC,5.12,89.0,-0.16,0.87,14.06,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.07,-16.40,71.44,43.32,97273.0,-26712.0,136160.0,-28.18,37.0,X-SC,4.74,90.0,-0.27,0.97,4.55,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-1.02,-12.09,37.31,20.71,124055.0,-45739.0,332498.0,-24.27,57.0,X-LC,4.46,1.0,-0.37,2.38,11.10,X40,ATH,IT
41,INFY,1972.00,-0.90,8.40,23.84,34.24,80861.0,26293.0,339181.0,-15.65,59.0,X-LC,4.01,2.0,0.33,2.42,17.38,X40,NTT,IT
75,TMPV,600.00,-0.62,-29.29,73.91,22.98,118060.0,-66163.0,159735.0,-26.74,29.0,X-LC,6.81,3.0,-0.56,1.14,0.47,XY24,NTT,AUTO
81,VBL,671.64,0.67,-2.88,39.69,35.67,121749.0,-9092.0,306750.0,-14.36,58.0,X-LC,4.11,4.0,-0.07,2.19,10.58,X40N,ATH,FMCG
43,ITC,452.00,0.06,-1.34,12.28,10.78,29116.0,-3216.0,237102.0,-41.15,47.0,X-LC,1.95,5.0,-0.11,1.69,3.83,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.95,-37.75,113.47,32.89,54536.0,-29144.0,48062.0,-693.69,53.0,L-MC,5.95,259.0,-0.53,0.34,32.36,XR,NTT,BANKS
6,ASIANTILES,137.00,2.16,-16.72,118.15,81.67,92302.0,-15687.0,78123.0,7022.22,46.0,L-SC,12.37,271.0,-0.17,0.56,51.29,XR,NTT,CERAMICS
50,MASFIN,398.61,-1.87,-1.90,24.41,22.05,23463.0,-1860.0,96120.0,-15.54,55.0,H-SC,6.90,164.0,-0.08,0.69,39.72,XR,ATH,FINANCE
13,BSOFT,831.70,-1.46,-22.33,94.14,50.79,102840.0,-31411.0,109242.0,-0.27,64.0,H-SC,6.34,151.0,-0.31,0.78,27.19,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.54,-13.04,15.01,0.02,22785.0,-22758.0,151801.0,131.58,50.0,M-SC,14.80,234.0,-1.00,1.08,32.04,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-1.46,-22.33,94.14,50.79,102840.0,-31411.0,109242.0,-0.27,64.0,H-SC,6.34,151.0,-0.31,0.78,27.19,XR,ATH,IT
84,WIPRO,420.00,-0.66,6.64,61.53,72.27,111189.0,11259.0,180707.0,-8.35,65.0,M-LC,2.46,99.0,0.10,1.29,13.23,XR,NTT,IT
41,INFY,1972.00,-0.90,8.40,23.84,34.24,80861.0,26293.0,339181.0,-15.65,59.0,X-LC,4.01,2.0,0.33,2.42,17.38,X40,NTT,IT
1,ABB,7934.00,-0.52,-3.54,50.91,45.57,128476.0,-9259.0,252360.0,-37.97,58.0,H-MC,4.55,120.0,-0.07,1.80,7.38,AR,NTT,ELECTRICAL
81,VBL,671.64,0.67,-2.88,39.69,35.67,121749.0,-9092.0,306750.0,-14.36,58.0,X-LC,4.11,4.0,-0.07,2.19,10.58,X40N,ATH,FMCG


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.24
1,25,44.86
2,50,76.63


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.95
MC    30.97
LC    24.05
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.72
X40      22.94
X40N     13.08
XR        9.77
XY25      9.09
AR        9.04
OX40N     7.57
SR        0.99
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.50
H-SC    23.80
X-LC    20.63
M-SC    11.69
X-SC     8.03
H-MC     4.81
L-SC     1.43
M-MC     1.32
M-LC     1.29
H-LC     1.12
L-LC     1.01
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.43,-6.99,41.39
IT,12.98,-15.73,73.92
FINANCE,9.48,-20.19,72.46
MISC,7.24,-28.18,80.61
ELECTRICAL,6.02,-11.24,51.99
PAINTS,5.84,-12.80,29.55
INSURANCE,4.77,-1.37,36.06
PHARMA,4.12,-1.82,33.87
AUTO,2.81,-33.60,79.91


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3183706.0,21
AR,1306296.0,10
XR,1295907.0,13
X40,1009745.0,14
X40N,853940.0,9
OX40N,730938.0,10
XY25,370879.0,6
SR,281428.0,2
MH,76704.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3648412.0,25
M-SC,1423586.0,15
X-MC,1396391.0,16
X-LC,839777.0,11
X-SC,793967.0,8
H-MC,404641.0,3
L-SC,261429.0,3
M-LC,111189.0,1
H-LC,76530.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1274438.0      6
           AR         894173.0      5
M-SC       XY24       819309.0      6
H-SC       XR         780240.0      7
X-MC       X40        484838.0      7
           XY24       400865.0      3
H-SC       OX40N      341429.0      4
X-LC       X40        336100.0      5
X-MC       X40N       333995.0      4
X-SC       X40N       308108.0      3
M-SC       OX40N      304289.0      5
X-SC       XY24       297052.0      3
H-SC       SR         281428.0      2
X-LC       X40N       211837.0      2
H-MC       AR         209338.0      2
X-LC       XY24       196739.0      2
H-MC       XY24       195303.0      1
X-SC       X40        188807.0      2
X-MC       XY25       176693.0      2
L-SC       XR         176209.0      2
M-SC       XR         173733.0      2
           AR         126255.0      2
M-LC       XR         111189.0      1
X-LC       XY25        95101.0      2
L-SC       OX40N       85220.0      1
H-SC       MH          76704.0      1
H-LC       AR          76530.0      1
M-MC       XY25        60615.0      1
L-MC       XR          54536.0      1
L-LC       XY25        38470.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
